In [1]:
import findspark

In [2]:
findspark.init("E:/Spark/spark-2.4.5-bin-hadoop2.7")

In [3]:
import pyspark

In [4]:
import pandas as pd
from pyspark.sql import *

In [5]:
MAX_MEMORY = '6g'
spark_session = SparkSession.builder \
    .master("local") \
    .appName("Task 3") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()

In [6]:
import pandas as pd
df = pd.read_csv("./train.csv", delimiter = ",")
df.head(5)

,movie_id,movie_name,plot,genre
0,23890098,Taxi Blues,"Shlykov, a hard-working taxi driver and Lyosha...","['World cinema', 'Drama']"
1,31186339,The Hunger Games,The nation of Panem consists of a wealthy Capi...,"['Action/Adventure', 'Action', 'Science Fictio..."
2,20663735,Narasimham,Poovalli Induchoodan is sentenced for six yea...,"['Musical', 'Action', 'Drama']"
3,2231378,The Lemon Drop Kid,"The Lemon Drop Kid , a New York City swindler,...",['Comedy']
4,595909,A Cry in the Dark,Seventh-day Adventist Church pastor Michael Ch...,"['Crime Fiction', 'World cinema', 'Drama']"


In [7]:
sdf1 = spark_session.createDataFrame(df)
sdf1.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [8]:
from pyspark.sql.functions import *
new= sdf1.withColumn('plot', regexp_replace('plot', '[{.*}]', ''))\
            .withColumn('plot', regexp_replace('plot', 'plot', '')) \
            .withColumn('plot', regexp_replace('plot', '[\(\)]',''))\
            .withColumn('plot', regexp_replace('plot', '[\[\]]', '')) \
            .withColumn('genre', regexp_replace('genre', '[\[\]]', '')) \
            .withColumn('plot', regexp_replace('plot',',',''))\
            .withColumn('genre', regexp_replace('genre', '[\']', ''))\
            .withColumn('genre', regexp_replace('genre',' ', ''))


In [9]:
sdf1 = new
#sdf1 = new1
sdf1.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hard-wo...|   Worldcinema,Drama|
|31186339|    The Hunger Games|The nation of Pan...|Action/Adventure,...|
|20663735|          Narasimham|Poovalli Induchoo...|Musical,Action,Drama|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|              Comedy|
|  595909|   A Cry in the Dark|Seventh-day Adven...|CrimeFiction,Worl...|
| 5272176|            End Game|The president is ...|Action/Adventure,...|
| 1952976|          Dark Water| The film opens i...|Thriller,Drama,Ho...|
|24225279|                Sing|The story begins ...|               Drama|
| 2462689|       Meet John Doe|Infuriated at bei...|Black-and-white,C...|
|20532852|Destination Meatball|A line of people ...|Animation,ShortFi...|
|15401493|    Husband for Hire|Lola  a

In [10]:
import pyspark.sql.types as types
sdf2 = spark_session.read.csv("./mapping.csv", sep = ',', escape='"', enforceSchema='true', header = 'true')
sdf2.show()

+---+----------------+
|_c0|               0|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [11]:
sdf2 = sdf2.withColumnRenamed('_c0','id')\
    .withColumnRenamed('0', 'genres')
sdf2.show()


+---+----------------+
| id|          genres|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [12]:
sdf2 = sdf2.withColumn('genres', regexp_replace('genres',' ',''))
sdf2.show()

+---+----------------+
| id|          genres|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|     RomanceFilm|
|  3|        Thriller|
|  4|          Action|
|  5|     Worldcinema|
|  6|    CrimeFiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|      FamilyFilm|
| 13|       ShortFilm|
| 14|   Romanticdrama|
| 15|       Animation|
| 16|         Musical|
| 17|  ScienceFiction|
| 18|         Mystery|
| 19|  Romanticcomedy|
+---+----------------+



In [13]:
vals = sdf2.select('genres').collect()
print(vals[0])

Row(genres='Drama')


In [14]:
genre_list = []
for i in range(len(vals)):
    genre_list.append(vals[i].asDict()['genres'])
genre_list

['Drama',
 'Comedy',
 'RomanceFilm',
 'Thriller',
 'Action',
 'Worldcinema',
 'CrimeFiction',
 'Horror',
 'Black-and-white',
 'Indie',
 'Action/Adventure',
 'Adventure',
 'FamilyFilm',
 'ShortFilm',
 'Romanticdrama',
 'Animation',
 'Musical',
 'ScienceFiction',
 'Mystery',
 'Romanticcomedy']

In [55]:
train_genre = sdf1.select('genre').collect()
v = train_genre[0].asDict()['genre']
v = v.split(",")
print(v)

['Worldcinema', 'Drama']


In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import numpy as np

def generate_one_hot_encoding(genre):
    v = genre.split(",")
    encoding = np.zeros(len(genre_list))
    for i in range(len(v)):
        for j in range(len(genre_list)):
            if v[i] == genre_list[j]:
                encoding[j] = 1 
            else:
                pass
    
    
    encoding = encoding.tolist()
    return encoding

In [16]:
import pyspark.sql.types as types
udffunc = udf(generate_one_hot_encoding, types.ArrayType(FloatType()))
new_sdf = sdf1.withColumn('encoding', udffunc('genre')) 
new_sdf.show()

+--------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|            encoding|
+--------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hard-wo...|   Worldcinema,Drama|[1.0, 0.0, 0.0, 0...|
|31186339|    The Hunger Games|The nation of Pan...|Action/Adventure,...|[1.0, 0.0, 0.0, 0...|
|20663735|          Narasimham|Poovalli Induchoo...|Musical,Action,Drama|[1.0, 0.0, 0.0, 0...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|              Comedy|[0.0, 1.0, 0.0, 0...|
|  595909|   A Cry in the Dark|Seventh-day Adven...|CrimeFiction,Worl...|[1.0, 0.0, 0.0, 0...|
| 5272176|            End Game|The president is ...|Action/Adventure,...|[1.0, 0.0, 0.0, 1...|
| 1952976|          Dark Water| The film opens i...|Thriller,Drama,Ho...|[1.0, 0.0, 0.0, 1...|
|24225279|                Sing|The story begins ..

In [17]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [18]:
tokenizer = Tokenizer(inputCol='plot', outputCol='plottokens')
new_sdf = tokenizer.transform(new_sdf)
remover = StopWordsRemover(inputCol = 'plottokens', outputCol = 'plottokensclean')
new_sdf = remover.transform(new_sdf)

In [19]:
new_sdf = new_sdf.drop('plottokens')

In [20]:
new_sdf.schema

StructType(List(StructField(movie_id,LongType,true),StructField(movie_name,StringType,true),StructField(plot,StringType,true),StructField(genre,StringType,true),StructField(encoding,ArrayType(FloatType,true),true),StructField(plottokensclean,ArrayType(StringType,true),true)))

In [21]:
new_sdf.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|            encoding|     plottokensclean|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov a hard-wo...|   Worldcinema,Drama|[1.0, 0.0, 0.0, 0...|[shlykov, hard-wo...|
|31186339|    The Hunger Games|The nation of Pan...|Action/Adventure,...|[1.0, 0.0, 0.0, 0...|[nation, panem, c...|
|20663735|          Narasimham|Poovalli Induchoo...|Musical,Action,Drama|[1.0, 0.0, 0.0, 0...|[poovalli, induch...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|              Comedy|[0.0, 1.0, 0.0, 0...|[lemon, drop, kid...|
|  595909|   A Cry in the Dark|Seventh-day Adven...|CrimeFiction,Worl...|[1.0, 0.0, 0.0, 0...|[seventh-day, adv...|
| 5272176|            End Game|The president is ...|Action/Adventure,...

In [22]:
sdf3 = spark_session.read.csv("./test.csv", sep = ',', escape='"', enforceSchema='true', header = 'true')
sdf3.show()

+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|
+--------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|
|29062594|A la salida nos v...|A group of teenag...|
| 9252321|   Come Back, Africa|This story of a Z...|
|13455076|       A Merry Mixup|The Stooges play ...|
|24165951|        Getting Even|A soldier-of-fort...|
| 1925869|  River of No Return|Set in the Northw...|
|10799612|          Amici miei|Like in many othe...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|
|17124781|The Good, the Bad...|In the desert wil...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|
|19174305|              Tahaan|Tahaan  lives wit...|
|18392317|     Mysterious Mose|Betty is startled...|
|34420857|Kelviyum Naane Pa...|Nirmal ([[Karthik...|
| 4039635|   First on the Moon|A group of journa...|
| 8034072|  Journey of a Woman|Vaibhavari Sahay,...|
| 4016437|     Sophie's Choice|In 1947, the mo

In [23]:
new3 = sdf3.withColumn('plot', regexp_replace('plot', '[{.*}]', ''))\
            .withColumn('plot', regexp_replace('plot', 'plot', '')) \
            .withColumn('plot', regexp_replace('plot', '[\(\)]',''))\
            .withColumn('plot', regexp_replace('plot', '[\[\]]',''))
sdf3 = new3
sdf3.show()

+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|
+--------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|
|29062594|A la salida nos v...|A group of teenag...|
| 9252321|   Come Back, Africa|This story of a Z...|
|13455076|       A Merry Mixup|The Stooges play ...|
|24165951|        Getting Even|A soldier-of-fort...|
| 1925869|  River of No Return|Set in the Northw...|
|10799612|          Amici miei|Like in many othe...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|
|17124781|The Good, the Bad...|In the desert wil...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|
|19174305|              Tahaan|Tahaan  lives wit...|
|18392317|     Mysterious Mose|Betty is startled...|
|34420857|Kelviyum Naane Pa...|Nirmal Karthik  t...|
| 4039635|   First on the Moon|A group of journa...|
| 8034072|  Journey of a Woman|Vaibhavari Sahay,...|
| 4016437|     Sophie's Choice|In 1947, the mo

In [24]:
sdf3 = tokenizer.transform(sdf3)
sdf3 = remover.transform(sdf3)
sdf3.show()

+--------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|          plottokens|     plottokensclean|
+--------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|
|24165951|        Getting Even|A soldier-of-fort...|[a, soldier-of-fo...|[soldier-of-fortu...|
| 1925869|  River of No Return|Set in the Northw...|[set, in, the, no...|[set, northwester...|
|10799612|          Amici miei|Like in many othe...|[like, in, many, ...|[like, many, moni...|
|28238240|Mickey's Big Game...|Mickey and the Sc..

In [25]:
sdf3 = sdf3.drop('plottokens')
sdf3.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|     plottokensclean|
+--------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[film, based, eve...|
|29062594|A la salida nos v...|A group of teenag...|[group, teenagers...|
| 9252321|   Come Back, Africa|This story of a Z...|[story, zulu, fam...|
|13455076|       A Merry Mixup|The Stooges play ...|[stooges, play, t...|
|24165951|        Getting Even|A soldier-of-fort...|[soldier-of-fortu...|
| 1925869|  River of No Return|Set in the Northw...|[set, northwester...|
|10799612|          Amici miei|Like in many othe...|[like, many, moni...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|[mickey, scorpion...|
|17124781|The Good, the Bad...|In the desert wil...|[desert, wilderne...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|[bimbo, koko, sig...|
|19174305|              Tahaan|Tahaan 

In [28]:
for i in range(len(genre_list)):
    new_sdf = new_sdf.withColumn("col{}".format(i),new_sdf['encoding'].getItem(i))


In [1]:
# df['col_name'] creates a Column Object , toaccess the elements we need to use getItem()

In [26]:
from pyspark.ml.feature import Word2Vec
from pyspark.ml import Pipeline

In [27]:
def make_and_fit(sdf, inputcol):
    w2v_model = Word2Vec(vectorSize=300, minCount=0, inputCol=inputcol, outputCol='plotvectors')
    doc2vec_pipeline = Pipeline(stages =[w2v_model] )
    doc2vec_model = doc2vec_pipeline.fit(sdf)
    return doc2vec_model


In [30]:
doc2vec_model = make_and_fit(new_sdf, inputcol="plottokensclean")
new_sdf = doc2vec_model.transform(new_sdf)

In [31]:
new_sdf.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------------------+
|movie_id|          movie_name|                plot|               genre|            encoding|     plottokensclean|col0|col1|col2|col3|col4|col5|col6|col7|col8|col9|col10|col11|col12|col13|col14|col15|col16|col17|col18|col19|         plotvectors|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------------------+
|23890098|          Taxi Blues|Shlykov a hard-wo...|   Worldcinema,Drama|[1.0, 0.0, 0.0, 0...|[shlykov, hard-wo...| 1.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|  0.0|[0.05814210692707...|
|31186339|  

In [32]:
d2v_model = make_and_fit(sdf3, inputcol = "plottokensclean")

In [33]:
sdf3 = d2v_model.transform(sdf3)

In [34]:
sdf3.show()

+--------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|     plottokensclean|         plotvectors|
+--------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[film, based, eve...|[0.01070872802175...|
|29062594|A la salida nos v...|A group of teenag...|[group, teenagers...|[0.00521442606651...|
| 9252321|   Come Back, Africa|This story of a Z...|[story, zulu, fam...|[0.01173783440051...|
|13455076|       A Merry Mixup|The Stooges play ...|[stooges, play, t...|[0.01266001568688...|
|24165951|        Getting Even|A soldier-of-fort...|[soldier-of-fortu...|[0.02019967997912...|
| 1925869|  River of No Return|Set in the Northw...|[set, northwester...|[0.01340642463277...|
|10799612|          Amici miei|Like in many othe...|[like, many, moni...|[0.01339420602512...|
|28238240|Mickey's Big Game...|Mickey and the Sc..

In [35]:
from pyspark.ml.classification import RandomForestClassifier

In [36]:
def fit_and_classify(sdf, labelcol, featurescol):
    rf_classifier = RandomForestClassifier(labelCol = labelcol, featuresCol= featurescol,numTrees=3,maxDepth=2)
    rf_classifier_pipeline = Pipeline(stages=[rf_classifier])
    rf_model = rf_classifier_pipeline.fit(sdf)
    return rf_model

In [72]:
# Code fragment for testing undersampling
l = []
for i in range(20):
    filter_df = new_sdf.select('movie_id','movie_name', 'plot','genre','plotvectors','col{}'.format(i))
#Dividing the filtered_df to check for the ratio
    df1 = filter_df.filter(filter_df['col{}'.format(i)] == 1.0)
    df2 = filter_df.filter(filter_df['col{}'.format(i)] == 0.0)
    if df2.count() > df1.count():
        ratio = df2.count() / df1.count()
        sampled_df = df2.sample(False, 1/ratio)
        final_df = sampled_df.union(df1)
    else:
        ratio = df1.count() / df2.count()
        sampled_df = df1.sample(False, 1/ratio)
        final_df = sampled_df.union(df2)
    rf_model = fit_and_classify(final_df, labelcol='col{}'.format(i), featurescol='plotvectors')
    predictions = rf_model.transform(sdf3)
    l.append(predictions.select('movie_id', 'prediction'))       

In [73]:
len(l)

20

In [74]:
for i in range(len(l)):
    l[i] = l[i].withColumnRenamed('prediction', 'prediction{}'.format(i))

In [76]:
l[18].show()

+--------+------------+
|movie_id|prediction18|
+--------+------------+
| 1335380|         0.0|
|29062594|         0.0|
| 9252321|         0.0|
|13455076|         1.0|
|24165951|         1.0|
| 1925869|         1.0|
|10799612|         1.0|
|28238240|         0.0|
|17124781|         1.0|
|28207941|         1.0|
|19174305|         1.0|
|18392317|         1.0|
|34420857|         1.0|
| 4039635|         1.0|
| 8034072|         0.0|
| 4016437|         0.0|
| 1520023|         1.0|
|24589422|         0.0|
|35068740|         1.0|
|21132951|         0.0|
+--------+------------+
only showing top 20 rows



In [77]:
joined_df = l[0]
for i in range(1,len(l)):
    joined_df = joined_df.join(l[i], on =['movie_id'], how = 'inner' )

In [78]:
from pyspark.ml.feature import VectorAssembler

In [79]:
cols = []
for i in range(len(l)):
    cols.append('prediction{}'.format(i))

In [80]:
vecAssembler = VectorAssembler(inputCols = cols, outputCol = 'predictions')
joined_df = vecAssembler.transform(joined_df)

In [81]:
joined_df.head(5)

[Row(movie_id='1335380', prediction0=0.0, prediction1=1.0, prediction2=0.0, prediction3=1.0, prediction4=0.0, prediction5=1.0, prediction6=0.0, prediction7=1.0, prediction8=0.0, prediction9=1.0, prediction10=1.0, prediction11=1.0, prediction12=0.0, prediction13=0.0, prediction14=0.0, prediction15=0.0, prediction16=0.0, prediction17=0.0, prediction18=0.0, prediction19=1.0, predictions=SparseVector(20, {1: 1.0, 3: 1.0, 5: 1.0, 7: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 19: 1.0})),
 Row(movie_id='29062594', prediction0=1.0, prediction1=1.0, prediction2=0.0, prediction3=0.0, prediction4=0.0, prediction5=1.0, prediction6=0.0, prediction7=1.0, prediction8=0.0, prediction9=0.0, prediction10=0.0, prediction11=0.0, prediction12=0.0, prediction13=0.0, prediction14=0.0, prediction15=0.0, prediction16=0.0, prediction17=0.0, prediction18=0.0, prediction19=1.0, predictions=SparseVector(20, {0: 1.0, 1: 1.0, 5: 1.0, 7: 1.0, 19: 1.0})),
 Row(movie_id='9252321', prediction0=0.0, prediction1=1.0, prediction2=0.0,

In [82]:
from pyspark.sql.types import *
vector_udf = udf(lambda vector: vector.toArray().tolist(),ArrayType(DoubleType()))
joined_df = joined_df.withColumn('predictions',vector_udf('predictions'))   

In [83]:
joined_df.show()

+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+--------------------+
|movie_id|prediction0|prediction1|prediction2|prediction3|prediction4|prediction5|prediction6|prediction7|prediction8|prediction9|prediction10|prediction11|prediction12|prediction13|prediction14|prediction15|prediction16|prediction17|prediction18|prediction19|         predictions|
+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+--------------------+
| 1335380|        0.0|        1.0|        0.0|        1.0|        0.0|        1.0|        0.0|        1.0|        0.0|        1.0|         1.0|         1.

In [97]:
def get_prediction_string(prediction):
    predictions = ""
    for i in range(len(prediction)):
        predictions += str(int(prediction[i]))+" "
    predictions = predictions.rstrip(" ")
    return predictions

In [98]:
from pyspark.sql import types
udffunc = udf(get_prediction_string, types.StringType())
new_df = joined_df.withColumn('predictions', udffunc('predictions')) 

In [87]:
new_df.show()

+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+--------------------+
|movie_id|prediction0|prediction1|prediction2|prediction3|prediction4|prediction5|prediction6|prediction7|prediction8|prediction9|prediction10|prediction11|prediction12|prediction13|prediction14|prediction15|prediction16|prediction17|prediction18|prediction19|         predictions|
+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+--------------------+
| 1335380|        0.0|        1.0|        0.0|        1.0|        0.0|        1.0|        0.0|        1.0|        0.0|        1.0|         1.0|         1.

In [99]:
drop_cols = []
for i in range(len(l)):
    drop_cols.append('prediction{}'.format(i))

In [100]:
new_df = new_df.drop(*drop_cols)

In [101]:
new_df.show()

+--------+--------------------+
|movie_id|         predictions|
+--------+--------------------+
| 1335380|0 1 0 1 0 1 0 1 0...|
|29062594|1 1 0 0 0 1 0 1 0...|
| 9252321|0 1 0 0 0 1 0 1 0...|
|13455076|0 0 0 0 0 1 0 0 0...|
|24165951|0 0 0 1 0 0 0 1 0...|
| 1925869|0 1 0 1 0 1 0 1 0...|
|10799612|0 1 0 0 1 0 0 1 0...|
|28238240|0 0 0 0 0 0 0 1 1...|
|17124781|0 0 0 1 0 1 0 1 0...|
|28207941|0 0 0 1 0 0 0 1 1...|
|19174305|0 1 0 0 0 1 0 1 1...|
|18392317|0 0 0 0 0 0 0 1 1...|
|34420857|0 0 0 0 1 1 0 1 0...|
| 4039635|0 0 0 1 0 0 0 1 0...|
| 8034072|0 1 1 0 0 1 0 1 1...|
| 4016437|0 1 0 0 0 1 0 1 0...|
| 1520023|0 1 0 1 0 0 0 1 1...|
|24589422|0 1 0 0 0 1 0 1 1...|
|35068740|0 1 0 0 0 1 0 1 1...|
|21132951|0 1 0 0 0 0 0 1 0...|
+--------+--------------------+
only showing top 20 rows



In [102]:
new_df.select('*').toPandas().to_csv("submission.csv", index = False)